In [57]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

: 

In [33]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [34]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [37]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [38]:
DTEmlp4b.train_DTE()

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 4 models
Model 1:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 14.20it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 15.36it/s]


./saved_models/model_2.pth
Model 3:


...

Training Process: 100%|██████████| 10/10 [00:00<00:00, 15.90it/s]


./saved_models/model_8.pth
Model 6:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 16.92it/s]

./saved_models/model_6.pth
 Maximum accuracy: 0.8860243055555556, F1: 0.6316220561576695, Test: 1.0076692238450051, Train: 1.029837812980016 
 



,DTE_acc,DTE_f1,DTE_test,DTE_train
0,0.886024,0.631622,1.007669,1.029838


In [39]:
DTEmlp4b.single_model()

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process: 100%|██████████| 70/70 [00:06<00:00, 10.30it/s]

 Maximum accuracy: 0.871999007936508, F1: 0.767410636902852, Test: 0.877471253488745, Train: 0.8809655171348935 
 



,single_acc,single_f1,single_test,single_train
0,0.871999,0.767411,0.877471,0.880966


In [40]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [53]:
output_directory = './fixed.csv'
run_experiment(DTEmlp4b, trails=100, output_directory=output_directory)

Trail 0: 

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 4 models
Model 1:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 14.48it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 17.16it/s]


./saved_models/model_2.pth
Model 3:


...

./saved_models/model_8.pth
Model 6:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.53it/s]


./saved_models/model_6.pth
 Maximum accuracy: 0.8403645833333332, F1: 0.7311975154810023, Test: 0.9795791894197464, Train: 0.9657110516230265 
 

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process: 100%|██████████| 70/70 [00:07<00:00,  8.96it/s]

 Maximum accuracy: 0.7571056547619048, F1: 0.6476700577168659, Test: 0.9055356202380997, Train: 0.9424078718821207 
 



In [54]:
df = pd.read_csv("fixed.csv")

In [55]:
print(df.columns)

Index(['DTE_acc', 'DTE_f1', 'DTE_test', 'DTE_train', 'single_acc', 'single_f1',
       'single_test', 'single_train'],
      dtype='object')


In [56]:
print(f'single acc mean: {df["single_acc"].mean()}  DTE acc mean: {df["DTE_acc"].mean()}')
print(f'single acc var: {df["single_acc"].var()}  DTE acc var: {df["DTE_acc"].var()}')
print(f'single loss mean: {df["single_test"].mean()}  DTE loss mean: {df["DTE_test"].mean()}')
print(f'single loss var: {df["single_test"].var()}  DTE loss var: {df["DTE_test"].var()}')

single acc mean: 0.8066558779761905  DTE acc mean: 0.8575434992283952
single acc var: 0.004676934480984211  DTE acc var: 0.006216706607605751
single loss mean: 0.8932053092207226  DTE loss mean: 0.9822792329341173
single loss var: 0.001863120451107072  DTE loss var: 0.0008684946649726689


In [ ]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [ ]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)